# **ĐỒ ÁN: NHẬP MÔN KHOA HỌC DỮ LIỆU** #

### Nhóm 7 ###
|<div style="width:150px">MSSV</div>|<div style="width:290px">Tên</div>|
|---------- |:-------------:|
| 20120041  |  Trần Kim Bảo |
| 20120053  |    Nguyễn Thành Đạt   | 
| 20120071  | Nguyễn Thị Bích Hà |
| 20120113  | Lê Nguyên Khang |

## <font color='orange'> Import thư viện </font> ##

In [1]:
!pip install scrapy
!pip install spider3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
import re
import pandas as pd
import numpy as np
import scrapy
from pandas.testing import assert_frame_equal # to compare two dataframes
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import thêm tại đây

## <font color='orange'> Mô tả trang web </font> ##

Trong đồ án này nhóm mình xin được giới thiệu đến mọi người danh sách phim hay nhất mọi thời đại *(Thời gian tính đến ngày 19/11/2022)* được trang IMDb tổng hợp. **IMDb** (<font color='blue'>https://www.imdb.com/</font>) là một trang cơ sở dữ liệu trực tuyến về điện ảnh thế giới, cung cấp thông tin về phim, diễn viên, đạo diễn, nhà làm phim,.. và tất cả những người, công ty trong lĩnh vực sản xuất phim, phim truyền hình và cả trò chơi video.

Những thông tin được thu thập tại website: <font color='blue'>https://www.imdb.com/list/ls006266261/</font>

### Tạo một project mới với scrapy ###

In [3]:
!scrapy startproject film_crawler

Error: Module 'film_crawler' already exists


In [7]:
cd film_crawler/film_crawler

/home/nguyendat/Documents/NguyenDat/Nam3Ky1/NhapMonKhoaHocDuLieu/Project/film_crawler/film_crawler


## <font color='orange'> Thu thập thông tin từ web </font> ##
**<h5>1. Thu thập url, id, votes, gross của các phim </h5>**

In [8]:
%%writefile spiders/collect_film_urls.py
import scrapy

class collect_film_url(scrapy.Spider):
  name='films_urls' 
  
  def start_requests(self):
    # urls=[]
    urls=['https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1']
    # for i in range(1,11):
    #   urls.append(f'https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page={i}')
    # for url in urls:
    self.page=1
    yield scrapy.Request(url=urls[0], callback=self.parse)
        
  def parse(self, response):
    top=response.css('div.lister-list span.lister-item-index.unbold.text-primary::text').getall()
    id=response.css('div.lister-list img::attr(data-tconst)').getall()
    url=response.css('div.lister-list div.lister-item.mode-detail h3.lister-item-header a::attr(href)').re(r'.*/title/.*\d{4,10}.')
    for i in range(len(top)):
      yield{
        'Top': top[i],
        'ID':id[i],
        'URL':url[i],
        'Votes':response.xpath(f'//*[@id="main"]/div/div[4]/div[3]/div[{i+1}]/div[2]/p[4]/span[2]/@data-value').get(),
        'Gross':response.xpath(f'//*[@id="main"]/div/div[4]/div[3]/div[{i+1}]/div[2]/p[4]/span[5]/@data-value').get()
      }
    if self.page<11:
      self.page+=1
      next_page_url=f'https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page={self.page}'
      yield scrapy.Request(url=next_page_url, callback=self.parse)
      

Overwriting spiders/collect_film_urls.py


- **Xuất các thông tin đã thu thập ở trên ra file json**

In [9]:
!scrapy crawl films_urls -o dataset/films_urls.json

2022-11-27 14:18:26 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: film_crawler)
2022-11-27 14:18:26 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.13, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.1.0, Python 3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0], pyOpenSSL 21.0.0 (OpenSSL 3.0.2 15 Mar 2022), cryptography 3.4.8, Platform Linux-5.15.0-53-generic-x86_64-with-glibc2.35
2022-11-27 14:18:26 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-11-27 14:18:26 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'film_crawler',
 'NEWSPIDER_MODULE': 'film_crawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['film_crawler.spiders']}
2022-11-27 14:18:26 [scrapy.extensions.telnet] INFO: Telnet Password: f19aeba5942fccfa
2022-11-27 14:18:26 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsag

In [15]:
%%writefile spiders/collect_data_films.py
import scrapy
import pandas as pd

class collect_data_film(scrapy.Spider):
  name='data_film'
  def __init__(self):
    try:
      df = pd.read_json('./dataset/films_urls.json', encoding='utf-8-sig')
      self.urls=df['URL']
      self.url_count = 1
    except IOError:
      print("File not found")
  def start_requests(self):
    # urls=[]
    first_url=['https://www.imdb.com/title/tt0068646/?ref_=ttls_li_tt']
    yield scrapy.Request(url=first_url[0], callback=self.parse)

  def parse(self, response):
    name=response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/text()').get()
    published_year=response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[1]/span/text()').get()
    age_rated=response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[2]/span/text()').get()
    duration=response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[3]/text()').getall()
    genres=response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/div/div[2]//text()').getall()
    
    yield {
      'Name':name,
      'Published Year':published_year,
      'Rated':age_rated,
      'Duration':duration,
      'Genres':genres
    }
    
    # print(name)
    if self.url_count<1000:
      next_page_url=f'https://www.imdb.com{self.urls[self.url_count]}?ref_=ttls_li_tt'
      self.url_count+=1
      yield scrapy.Request(url=next_page_url, callback=self.parse)

    

Overwriting spiders/collect_data_films.py


In [16]:
!scrapy crawl data_film -o dataset/data_film.json

2022-11-27 14:24:57 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: film_crawler)
2022-11-27 14:24:57 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.13, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.1.0, Python 3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0], pyOpenSSL 21.0.0 (OpenSSL 3.0.2 15 Mar 2022), cryptography 3.4.8, Platform Linux-5.15.0-53-generic-x86_64-with-glibc2.35
2022-11-27 14:24:57 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-11-27 14:24:57 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'film_crawler',
 'NEWSPIDER_MODULE': 'film_crawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['film_crawler.spiders']}
2022-11-27 14:24:57 [scrapy.extensions.telnet] INFO: Telnet Password: d280ed164e18e5dd
2022-11-27 14:24:57 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsag

- **Đọc file film_urls vào dataframe**

In [7]:
df_url=pd.read_json('dataset/films_urls.json',encoding='utf-8-sig')
df_url.head()

,Top,ID,URL,Votes,Gross
0,1.,tt0068646,/title/tt0068646/,1846269,"134,966,411"
1,2.,tt0099685,/title/tt0099685/,1154676,"46,836,394"
2,3.,tt0110912,/title/tt0110912/,2039241,"107,928,762"
3,4.,tt0114814,/title/tt0114814/,1081964,"23,341,568"
4,5.,tt0078788,/title/tt0078788/,665967,"83,471,511"


### Ý nghĩa mỗi dòng: ###

Là thông tin mang một vài thông tin sơ bộ: top, id, url, votes, gross (*Nhóm chỉ lấy một vài thông tin, vì có một số thông tin chỉ xuất hiện ở phần tổng quát, và nhóm quyết định sẽ lấy thông tin chi tiết hơn ở lần thu thập chi tiết của từng phim*) 


### Ý nghĩa mỗi cột: ###

- Top: thể hiện vị thứ của phim trong danh sách
- ID: là duy nhất và mang tính định danh cho phim
- URL: địa chỉ truy cập vào trang web mô tả chi tiết của phim
- Votes: số lượt bình chọn cho phim
- Gross: Thu nhập của phim tính đến thời điểm hiện tại

**<h5>2. Thu thập thông tin chi tiết của từng phim </h5>**

## <font color='orange'> Tiền xử lý dữ liệu </font> ##
**<h5>1. Tiền xử lý dataframe URL </h5>**

In [8]:
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Top     1000 non-null   object
 1   ID      1000 non-null   object
 2   URL     1000 non-null   object
 3   Votes   1000 non-null   int64 
 4   Gross   937 non-null    object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


- Ta thấy ở cột Top, Gross đang ở kiểu object, nhưng đúng là ở dạng numberic, ta thực hiện tiền xử lý về đúng kiểu dữ liệu

In [9]:

df_url['Top']=df_url['Top'].replace(",",'',regex=True)
df_url['Top']=df_url['Top'].astype(float)

df_url['Gross']=df_url['Gross'].replace([",",None],['',np.nan],regex=True)
df_url['Gross']=df_url['Gross'].astype(float)
display(df_url)

,Top,ID,URL,Votes,Gross
0,1.0,tt0068646,/title/tt0068646/,1846269,134966411.0
1,2.0,tt0099685,/title/tt0099685/,1154676,46836394.0
2,3.0,tt0110912,/title/tt0110912/,2039241,107928762.0
3,4.0,tt0114814,/title/tt0114814/,1081964,23341568.0
4,5.0,tt0078788,/title/tt0078788/,665967,83471511.0
...,...,...,...,...,...
995,996.0,tt0187738,/title/tt0187738/,220357,82348319.0
996,997.0,tt0106856,/title/tt0106856/,191362,40903593.0
997,998.0,tt0120663,/title/tt0120663/,342222,55691208.0
998,999.0,tt0093437,/title/tt0093437/,145605,32222567.0


<h1> https://www.imdb.com +url</h1>